# Value at Risk and Conditional Value at Risk

In this chapter, we'll be looking at two important statistics of risk management when trading currencies: the Value at Risk (VaR) and the Conditional VaR (CVaR). As always, these notebooks are not concerned in teaching you the theoretical concepts in detail (there are enough webpages out there which will do a far better job). Instead, we brute force theory in your brain with raw mathematics and a hands-on practical example. Ahhh, the true data hacker way.

The main goal of this session is to show you why RISK is best seen as a flashing red light in your mind, deserving all your attention while trading. But what is risk? Between us, young padawan, we'll agree that risk is simply the amount of money you're about to lose. We need to somehow be able to estimate the risk we're about to take when we'll enter the market. Otherwise you'll go for coffee and come back with a brick in your window and your kidney gone. Stay safe out there kids, markets can move fast and banks are mafias.

Let's start off by looking at the amount a currency value can move in a given time: the return distribution. 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, probplot

sns.set(style="ticks", context="talk")
plt.style.use("dark_background")